# neuralmt: default program

In [1]:
from default import *
import os, sys

## Run the default solution on dev

In [2]:
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [06:11,  3.51it/s]

i had my own , , i had my own . . 
<unk> was <unk> <unk> , and <unk> <unk> . 
<unk> was a <unk> <unk> . 
and she was , and she was <unk> , and she was <unk> , and she was <unk> , and she <unk> , and she was <unk> , and she <unk> , and she <unk> , and she <unk> . 
i was , i was , i was , i was . 
and so , a first - <unk> , , a <unk> , a <unk> . 
and i was going to talk about the , and i wanted to talk about the <unk> . 
i i , i i , i . 
i did n't have it . 
and , the the , , , the , , , was to the <unk> , , and the <unk> , and the <unk> , and the <unk> , and the <unk> , and it was <unk> to 
i was , , " , " , " , " , " and , " 
later came later , later came later later came later later came later later came later later later came later later later later later later later later later later later later later later later later later later later later later later later later later later later later . 
and i did n't like , like , and i did n't have a <unk> like 
i was to <unk> my <unk> , <unk

## Evaluate the default output

In [4]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))

BLEU = 2.49 28.3/5.4/1.3/0.4 (BP = 0.854 ratio = 0.864 hyp_len = 21503 ref_len = 24902)


# Baseline: fix the attention module

In [2]:
import os, sys
sys.path.append("..")
from neuralmt import *

In [3]:
class AttentionModule(nn.Module):
    def __init__(self, attention_dim):
        """
        You shouldn't deleted/change any of the following defs, they are
        essential for successfully loading the saved model.
        """
        super(AttentionModule, self).__init__()
        self.W_enc = nn.Linear(attention_dim, attention_dim, bias=False)
        self.W_dec = nn.Linear(attention_dim, attention_dim, bias=False)
        self.V_att = nn.Linear(attention_dim, 1, bias=False)
        return

    # Start working from here, both 'calcAlpha' and 'forward' need to be fixed
    def calcAlpha(self, decoder_hidden, encoder_out):
        """
        param encoder_out: (seq, batch, dim),
        param decoder_hidden: (seq, batch, dim)
        """
        seq, batch, dim = encoder_out.shape
        #scores = torch.Tensor([seq * [batch * [1]]]).permute(2, 1, 0)
        #alpha = torch.nn.functional.softmax(scores, dim=1)
        scores = self.W_enc(encoder_out.permute(1, 0, 2)) + self.W_dec(decoder_hidden.permute(1, 0, 2))
        alpha = torch.nn.functional.softmax(self.V_att(torch.tanh(scores)), dim=1)
        return alpha

    def forward(self, decoder_hidden, encoder_out):
        """
        encoder_out: (seq, batch, dim),
        decoder_hidden: (seq, batch, dim)
        """
        alpha = self.calcAlpha(decoder_hidden, encoder_out)
        seq, _, dim = encoder_out.shape
        #context = (torch.sum(encoder_out, dim=0) / seq).reshape(1, 1, dim)
        context = (torch.sum(alpha.permute(1, 0, 2) * encoder_out, dim=0)).reshape(1, 1, dim)
        return context, alpha.permute(2, 0, 1)

## Run the solution on dev

In [5]:
model = Seq2Seq(build=False)
model.load(os.path.join('../data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('../data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [09:24,  2.31it/s]

when when i was in my 20s , i had my first <unk> . . 
i was a and i was particularly in berkeley . berkeley . 
she was a a woman woman named alex . 
when the came came up the first first , she came up and she a a a , and she fell into the couch in my office , <unk> her her and and told me , about about about . 
and when i heard this , i was was . 
my my , , had a a patient patient for a first patient . 
and i got a woman in the people who wanted to talk about about . 
i 'll leave that , i thought , . 
but i did n't have it . 
with the the stories , that alex opened into the , was easy easy to me just to with with head , while we were the problems . 
" 30 is the new 20 " , " alex , and as far as i did , i had right . 
work came later later , later came later later , later came later later later later came later later . 
people people in the the , and alex , and i did n't have time . 
but soon my my my was , my head , to ask . 
i i was going to . 
i said , " yeah , you know , you 're wit

## Evaluation

In [10]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('../data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results).score)

17.11393268339238


## Analysis

The baseline requires us to fix the attention module. To be specific, we take a probability distribution (softmax) over the hidden states of the encoder. calcAlpha returns the alpha vector which is the softmax and tanh applied to our score in alpha = torch.nn.functional.softmax(self.V_att(torch.tanh(scores)), dim=1). The scores are the encoder hidden state added by decoder hidden state in scores = self.W_enc(encoder_out.permute(1, 0, 2)) + self.W_dec(decoder_hidden.permute(1, 0, 2)). forward returns the context vector which is the sum of alpha vector weights multiplied by the decoder hidden state for each source side index with context = (torch.sum(alpha.permute(1, 0, 2) * encoder_out, dim=0)).reshape(1, 1, dim). self.W_enc, self.W_dec, and self.V_attr holds the hidden states and scores to the Torch linear layers. The attention output mostly contains information from the hidden states that received high attention. Attention mechanism can improve the score because it is developing a context vector. The context vector can be filtered specifically for each output time step. 

The score we got was 17.1139.

# Improvement - Beam Search

In [6]:
import os, sys
sys.path.append("..")
from neuralmt import *

In [7]:
'''
we use Beam Decoder instead of Greedy Decoder. The latter finds a local optimal in each step, 
while the former find the approximately global optimal without great cost of exhaustive search.
'''
def beamDecoder(decoder, encoder_out, encoder_hidden, maxLen,
                  eos_index, target_vocab_size):
    seq1_len, batch_size, _ = encoder_out.size()
    target_vocab_size = decoder.target_vocab_size

    outputs = torch.autograd.Variable(
        encoder_out.data.new(maxLen, batch_size, target_vocab_size))
    alphas = torch.zeros(maxLen, batch_size, seq1_len)
    # take what we need from encoder
    decoder_hidden = encoder_hidden[-decoder.n_layers:]
    # start token (ugly hack)
    output = torch.autograd.Variable(
        outputs.data.new(1, batch_size).fill_(eos_index).long())

    root = Node(None, 0, output, decoder_hidden)
    queue = deque()
    queue.append(root)
    # condidates = []
    
    for t in range(maxLen):
        score_list = []
        # node_list = []
        info_list = []

        for i in range(len(queue)):
            # Get the parent node
            parent = queue.popleft()
            output, decoder_hidden, alpha = decoder(
                parent.output, encoder_out, parent.decoder_hidden)
            score = output.squeeze()       # size([1, 1, target_vocab_size]) -> size([target_vocab_size])
            score = torch.nn.functional.softmax(score, dim=-1)
            score = parent.log_prob + torch.log(score)
            # save score and (parent, decoder_hidden) tuples
            score_list.append(score)
            info_list.append( (parent, decoder_hidden) )

        topk_scores = torch.stack(score_list).view(-1).data.topk(hp.topk)  
        # topk_scores(values=[log_prob1,...], index=[word_id1, ...])
        # values: topk_scores[0]
        # indices: topk_scores[1]

        topk_parent_index = topk_scores[1] // target_vocab_size
        topk_word_id = topk_scores[1] % target_vocab_size
        
        # Insert new nodes to the queue
        for j in range(hp.topk):
            # If node.word == '<EOS>', stop it
            # if sorted_list[k].output == eos_index:
            #     candidates.append(sorted_list[k])
            #     continue
            parent_index = topk_parent_index[j]
            parent_   = info_list[parent_index][0]
            decoder_h = info_list[parent_index][1]
            score_    = topk_scores[0][j]
            word_     = topk_word_id[j].reshape(1, 1)
            new_node = Node(parent_, score_, word_, decoder_h)

            queue.append(new_node)
    
    # Get the node with the best score and backtrace
    node = queue[0]
    outputs = []
    while node != None:
        outputs.append(node.output)
        node = node.parent
    # Flip the output list
    outputs = torch.flip(torch.LongTensor(outputs), [0])
    outputs = outputs[1:(hp.max_len+1)].reshape(hp.max_len, 1, 1)

    return outputs


def translate(model, test_iter):
    results = []
    for i, batch in tqdm(enumerate(test_iter)):
        output = model(batch.src)
        output = model.tgt2txt(output[:, 0].data).strip().split('<EOS>')[0]
        new_output = []
        prev = None
        for word in output.split(' '):
            # If repeated words, delete one
            if prev != None and prev == word:
                continue
            # OOV - replace with 'the' - most common word
            if word == '<unk>':
                word = 'the'
            new_output.append(word)
            prev = word
        results.append(' '.join(new_output))
    return results

## Run the solution on dev

In [8]:
model = Seq2Seq(build=False)
model.load(os.path.join('../data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('../data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [3:31:02,  9.70s/it]

when i was in my 20s , i had my first the . 
i 've been up and studied clinical in berkeley . berkeley . 
she was a woman named alex . 
as soon as the first arrived came , she came up , and she was a , and she fell into my office in my office , the her , and she told me to talk about the . 
and when i heard this , i was the . 
i mean , my the had a patient as a first patient . 
and i got a woman in the people who wanted to talk about . 
i 'll leave it , i thought . 
but i did n't have it . 
with the stories , the alex came into the , it was easy to me , just to deal with head , as we faced the problems . 
" 30 is the new 20 " , " alex , and as far as i did , she had right . 
work came later , later came later , later came later , later came later . 
people in the , like alex , and i did n't have time . 
but soon became my god , alex the in question . 
i was against . 
i said , " yeah , yeah , you 'll be close with your , you with a , but she 's not going to marry it . " 
and since my s

## Evaluation

In [9]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('../data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results).score)

20.102193188828192


## Analysis

We implement beam search to get approximate global optimal instead of local optimal in greedy search. The idea is: at each step, keep track of the k most probable partial translations (the top-k log probabilities). When the path with the best score is found, we backtrack along the path and output the whole sentence. This was shown in the beamDecoder function, where we create nodes containing the parent, log prob score, word, and hidden decoder and push them on the queue. The top-k probable partial translations will be added to the queue. We get the node with the best score then backtrace by traversing through the node's parents until there are no more nodes left to traverse and apply transformations to get the best word sequence for the outputs array.

When k is equal to 2, we got score 18.3622. It is a slight improvement compared to our baseline score.

Hoever, we found there were many 'unk' words in the text. It's hard to sparse them. The simple method we implemented was to replace "unk" with "the" that is the most common word in English. After that, we got score 18.3882. The replacement was shown in the translate function.
    
After our analysis, we thought k was still too small here and it should be larger. After tuning the value of k several times, we determined that k=40 in our implementation. Another important thing we did was to remove repeated words (also shown in the translate function). Eg. "beauty beauty" => "beauty".

When we finished these improvements, the final BLEU score is 20.1022.

From the neuralmt.py file, you can find the k value we defined from the hyperparameter class hp where we set hp.topk to 40 at the top of the file after the python imports.

## Remaining problem

In this language model, we still have a weak point which is the long runtime length. Each word takes 6-7 seconds to translate the single sentence because there is a lot of time spent on Beam Decoding and Translation. It is better than an exhaustive search, but still time-consuming. 

time/it $\propto$ k. The time of processing one sentence is basically depend on the parameter k. When k increases, we have more nodes to expand in each step. 

Also, replacing 'unk' by 'the' is not reasonable. We have to look for another good replacement method.